In [1]:
#########BLOQUE IMPORTACIÓN LIBRERIAS
from	pyspark.mllib.linalg	import	Vector,	Vectors
from	pyspark.mllib.regression	import	LabeledPoint
import math

#########BLOQUE DEFINICIÓN FUNCIONES

def correlation(size, dotProduct, ratingSum,rating2Sum,ratingNormSq,rating2NormSq):
    numerator = size * dotProduct - ratingSum * rating2Sum
    denominator = math.sqrt(size * ratingNormSq - ratingSum * ratingSum)*math.sqrt(size * rating2NormSq - rating2Sum * rating2Sum)
    #if denominator = 0:
     #   corr =0;
    #else:
     #   corr = numerator / denominator
    return  3


def cosineSimilarity(dotProduct,ratingNorm,rating2Norm): 
    if ratingNorm * rating2Norm == 0:
        cosi = 0
    else: 
        cosi = dotProduct / (ratingNorm * rating2Norm)
    
    return cosi


###CARGA DATOS, usamos otra fuente  QUE TIENE LA MISMA ESTRUCTURA (UserId,Publisher Id),con fuente
Logs = sc.textFile('Data/advertising.csv')
header = Logs.first()
#quito headers
Lorgs_norm = Logs.filter(lambda row: row != header).map(lambda l: l.split(','))
#Para el calculo , me quedo solo con aquellos usuarios que hayan recibido la impresion original
#agrupamos por publisher, y sacamos el numero de usuarios por publisher, de la forma (publisher, numero de usuarios)

Log2Rec = Lorgs_norm.map(lambda tokens: (int(tokens[2]),int(tokens[1]),1 if tokens[5] == "True" else 0)).cache()


In [2]:
#agrupamos por usuario, y sacamos el numero de interacciones por publisher, de la forma (usuario, numero de interacciones con publishers)
num_InterPerUser = Log2Rec.groupBy(lambda (x,y,z): x).map(lambda (x,iterator):(x,len(iterator)))

In [4]:
#filtro los que tengan menos de una interacción
num_InterPerUser_fil = num_InterPerUser.filter(lambda (x,y): y>2) 

In [8]:
num_InterPerUser_fil.take(2)

[(1048580, 3), (1048590, 4)]

In [9]:
num_InterPerUser_fil.count()

168498

In [24]:
num_InterPerUser_fil.count()

168498

In [5]:
#agrupamos por usuario, desde key y hacemos el join para tener ( usuario,publisher, imp, volumen de interacciones por usuario)
ints_sWithSize = Log2Rec.groupBy(lambda (x,y,z): x).join(num_InterPerUser_fil).flatMap(lambda(a,(b,c)):((x,c) for x in b))

In [26]:
ints_sWithSize.take(2)

[((1048580, 2134, 0), 3), ((1048580, 2429, 0), 3)]

In [6]:
ints_sWithSize_2 = ints_sWithSize.map(lambda((a,b,c),d):(b,(a,c,d)))

In [16]:
#HAgo pares usuario a usuario desde la interracción por publisher
ints_sWithSize_2.take(2)

[(2134, (1048580, 0, 3)), (2429, (1048580, 0, 3))]

In [27]:
ints_sWithSize_2.count()

696110

In [7]:
#reasigno key al publisher, y hago un join , de cara a ver cada una de las interecciones comunes entre usuarios, usuario a usuario
inter_pairs_pub = ints_sWithSize_2.join(ints_sWithSize_2)
#(publisher, usruo 1, usuario 2)

In [51]:
inter_pairs_pub.take(2)

[(2048, ((1310980, 0, 5), (1310980, 0, 5))),
 (2048, ((1310980, 0, 5), (1399116, 0, 3)))]

In [9]:
#me quito duplicidades y los que son iguales, buscamos tener copres entre peliculas para cada usuario
ratingPairs = inter_pairs_pub.map(lambda (x,((a,b,c),(a1,b1,c1))):(x,((int(a),int(b),c),(int(a1),int(b1),c1)))).filter(lambda (x,((a,b,c),(a1,b1,c1))):a<a1)

In [52]:
ratingPairs.take(2)

[(2048, ((1310980, 0, 5), (1399116, 0, 3))),
 (2048, ((1310980, 0, 5), (1340412, 0, 4)))]

In [10]:
#z el rating , (par)
vectorial_params = ratingPairs.map(lambda (a,((y,z,k),(y1,z1,k1))):((y,y1),(z*z1, z,z1,z*z,z1*z1,k,k1,1)) )

In [9]:
vectorial_params.take(4)

[((1310980, 1399116), (0, 0, 0, 0, 0, 5, 3, 1)),
 ((1310980, 1340412), (0, 0, 0, 0, 0, 5, 4, 1)),
 ((1310980, 1349928), (0, 0, 0, 0, 0, 5, 3, 1)),
 ((1310980, 1361340), (0, 0, 0, 0, 0, 5, 3, 1))]

In [15]:
vectorial_params_3 = vectorial_params.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4],x[5]+y[5],x[6]+y[6],x[7]+y[7])).cache()

In [54]:
vectorial_params_3.take(5)

[((1248672, 1286966), (0, 0, 0, 0, 0, 5, 7, 1)),
 ((1095255, 1155955), (0, 0, 0, 0, 0, 3, 4, 1)),
 ((1226657, 1269965), (0, 0, 0, 0, 0, 7, 3, 1)),
 ((1069756, 1397362), (0, 0, 0, 0, 0, 3, 3, 1)),
 ((1113240, 1319382), (0, 1, 0, 1, 0, 4, 4, 1))]

In [12]:
def correlation(size, dotProduct, ratingSum,rating2Sum,ratingNormSq,rating2NormSq):
    numerator = size * dotProduct - ratingSum * rating2Sum
    denominator = math.sqrt(size * ratingNormSq - ratingSum * ratingSum)*math.sqrt(size * rating2NormSq - rating2Sum * rating2Sum)
    if denominator == 0:
        corr =0 
    else:
        corr = numerator / denominator
    return  corr

def cosineSimilarity(dotProduct,ratingNorm,rating2Norm): 
    if ratingNorm * rating2Norm == 0:
        cosi = 0
    else: 
        cosi = dotProduct / (ratingNorm * rating2Norm)
    
    return cosi

In [16]:
simmil_metrics_2= vectorial_params_3.mapValues(lambda (a,b,c,d,e,f,g,h):(h,correlation(h,a,b,c,d,e))).cache()


In [13]:
simmil_metrics= vectorial_params_3.mapValues(lambda (a,b,c,d,e,f,g,h):(h,cosineSimilarity(a,math.sqrt(d),math.sqrt(e)))).cache()


In [17]:
simmil_metrics_2.count()

80443657

In [ ]:
#obtengo los items que guardan algún tipo de similaridad
item = 1111
sample = simmil_metrics.filter(lambda ((a,b),(x,y)): a==item or b==item).collect()

In [19]:
simmil_metrics_2.take(2)

[((1256224, 1292438), (1, 0)), ((1207951, 1310619), (1, 0))]

In [20]:
crit = simmil_metrics.filter(lambda ((a,b),(x,y)):y>0.5).cache()

In [ ]:
crit.take(5)